In [14]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model='gpt-4o-mini')


In [15]:
class sentimentSchema(BaseModel):
  sentiment: Literal["positive", "negative"] = Field("Sentiment of the review")


class diagnosisOutput(BaseModel):
  issueType: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
  tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
  urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')



structuredModelSentiment = model.with_structured_output(sentimentSchema)
structuredModelDiagnosis = model.with_structured_output(diagnosisOutput)

In [16]:
class reviewState(TypedDict):
  review: str
  sentiment: Literal["positive", 'negative']
  diagnosis: dict
  response: str

In [19]:
def sentimentAnalysis(state: reviewState):
  prompt = f'for the following review find the sentiment. Review - \n {state["review"]}'

  output = structuredModelSentiment.invoke(prompt).sentiment

  return {'sentiment':output}

def checkSentiment(state: reviewState) -> Literal["positiveResponse", "runDiagnosis"]:
  if(state["sentiment"] == "positive"):
    return "positiveResponse"
  else:
    return "runDiagnosis"

def positiveResponse(state: reviewState):
  prompt = f'write a warm thanks message to the review. \n\n review - {state["review"]}. \n\n Also kindly ask the user to leave a feedback to our website also.'

  response = model.invoke(prompt).content

  return {'response': response}

def runDiagnosis(state: reviewState):
  prompt = f'Diagnose this negative review:\n\n{state["review"]}\n Return issue type, tone, and urgency'

  output = structuredModelDiagnosis.invoke(prompt)

  return {'diagnosis' : output.model_dump()}

def negativeResponse(state: reviewState):
  diagnosis = state['diagnosis']
  prompt = f'You are a support assitant, user had a {diagnosis["issueType"]} isse, sounded {diagnosis["tone"]}, and marked urgency as - {diagnosis["urgency"]}. Write a empathic and helpful resolution message to the user.'

  response = model.invoke(prompt).content

  return {'response':response}

In [23]:
graph = StateGraph(reviewState)

graph.add_node('sentimentAnalysis', sentimentAnalysis)
graph.add_node('positiveResponse', positiveResponse)
graph.add_node('runDiagnosis', runDiagnosis)
graph.add_node('negativeResponse', negativeResponse)

graph.add_edge(START, 'sentimentAnalysis')
graph.add_conditional_edges('sentimentAnalysis', checkSentiment)


graph.add_edge('positiveResponse', END)
graph.add_edge('runDiagnosis', 'negativeResponse')
graph.add_edge('negativeResponse',END)

workflow = graph.compile()

In [25]:
negReview = "This phone is very much bad and I am feeling very angry because money is waste and mobile is not working like a mobile should work. Battery is dying like a fast runner and when I touch the screen it is not listening to my finger, so I am clicking and clicking but nothing happens. Camera is making me look like a ghost or a potato because everything is blurry and dark even when sun is in the sky. Also it is becoming very hot like a pressure cooker when I use only for five minutes and I think it will blast in my hand. Sound is very low like a mosquito is singing and I cannot hear my friend talking on the call. Do not buy this phone because it is a total headache and the box was also looking very old and dusty. Very worst experience and I want my money back immediately."

posReview =  "This mobile phone is an absolute powerhouse that excels in every department, making it one of the most well-rounded devices I have ever used. From the moment I started using it, the blazing-fast performance and seamless multitasking—likely thanks to its high-end processor—impressed me, as it handles even the most demanding tasks without a hint of lag. The display is incredibly vibrant and smooth, providing an immersive experience for both media consumption and daily scrolling, while the camera setup captures stunningly sharp photos with excellent color accuracy in various lighting conditions. Furthermore, the battery life is exceptionally reliable, easily lasting through a full day of heavy use, and the ultra-fast charging capability is a total game-changer that eliminates any battery anxiety. Overall, it delivers a premium, flagship-level experience across the board, and I couldn't be happier with this purchase."

intital_state = {'review' : negReview}

workflow.invoke(intital_state)

{'review': 'This phone is very much bad and I am feeling very angry because money is waste and mobile is not working like a mobile should work. Battery is dying like a fast runner and when I touch the screen it is not listening to my finger, so I am clicking and clicking but nothing happens. Camera is making me look like a ghost or a potato because everything is blurry and dark even when sun is in the sky. Also it is becoming very hot like a pressure cooker when I use only for five minutes and I think it will blast in my hand. Sound is very low like a mosquito is singing and I cannot hear my friend talking on the call. Do not buy this phone because it is a total headache and the box was also looking very old and dusty. Very worst experience and I want my money back immediately.',
 'sentiment': 'negative',
 'diagnosis': {'issueType': 'Performance', 'tone': 'angry', 'urgency': 'high'},
 'response': "Subject: We're Here to Help – Let’s Resolve Your Performance Issue\n\nHi [User's Name],\n